In [239]:
import numpy as np
import pandas as pd

In [240]:
X_train = pd.read_csv("../Data/df_train_vvfinal").drop("Unnamed: 0", axis = 1)
y_train = pd.read_csv("../Data/y_train_vfinal").drop("Unnamed: 0", axis = 1)

In [241]:
X_val = pd.read_csv("../Data/df_val_vvfinal").drop("Unnamed: 0", axis = 1)
y_val = pd.read_csv("../Data/y_val_vfinal").drop("Unnamed: 0", axis = 1)

In [242]:
X_test = pd.read_csv("../Data/df_test_vvfinal").drop("Unnamed: 0", axis = 1)
y_test = pd.read_csv("../Data/y_test_vfinal").drop("Unnamed: 0", axis = 1)

In [243]:
X_train_normalize = (X_train - X_train.mean())/X_train.std()
y_train_normalize = (y_train - y_train.mean())/y_train.std()

In [244]:
X_val_normalize = (X_val - X_val.mean())/X_val.std()
y_val_normalize = (y_val - y_val.mean())/y_val.std()

In [245]:
y_train_normalize

,price_doc
0,-0.266820
1,-0.235510
2,-0.298131
3,1.246515
4,1.921034
...,...
24372,-0.078842
24373,3.730471
24374,-0.032836
24375,1.330009


## Neural Network

In [254]:
class NeuralNetwork(): 
    def fit(self, X, y, n_hidden, nodes, activations, lr, validation_X, validation_y, batch_size = 0):
        self._lr = lr
        self._X = X.values
        self._y = y.values
        self._n_hidden = n_hidden
        self._nodes = nodes
        self._weights = self._generate_weights()
        self._biases = self._generate_bias()
        self._activations = activations
        self._forward_inputs = []
        self._val_X = validation_X.values
        self._val_y = validation_y.values
        self._batch_size = batch_size
        

        return self._train()
    
    def predict(self, X):
        pred = X
        weights = self._weights
        biases = self._biases
        activations = self._activations[1:-1]
    
        for idx, layer in enumerate(weights):
            if idx == (len(weights) - 1):
                pred = (pred @ layer) + biases[idx].T,
            else:
                weight_output = (pred @ layer) + biases[idx].T
                pred = self._activation(data = weight_output, activation = activations[idx])

        return pred[0]
        
    def _activation(self, data, activation = "relu"):
        if activation == "relu":
            def relu(data):
                return np.array([max(0,i) for i in data]).reshape(data.shape)
            return np.apply_along_axis(relu, 1, data)
        if activation == "sigmoid":
            def sigmoid(data):
                return (1/(1 + np.exp(-data))).reshape(data.shape)
            return np.apply_along_axis(sigmoid, 1, data)
    
    def _der_activation(self, points, activation = "relu"):
        if activation == "relu":
            def d_relu(point):
                return np.array([0 if y <= 0 else 1 for y in point])
            return np.apply_along_axis(d_relu, 1, points)
        if activation == "sigmoid":
            ## todo
            return
    
    def _loss_function(self, ypred, loss = "l2"):
        y = self._val_y
        if loss == "mse":
            return ((ypred - y) ** 2).mean()
        if loss == "l2":
            return (((ypred - y) ** 2)/2).mean()
    
    def _loss_jacobian(self, ypred, loss = "l2"):
        if self._batch_size > 0:
            y = self._batchy
        else:
            y = self._y
        if loss == "l2":
            return (ypred - y)/(len(ypred))
    
    def _generate_weights(self):
        hidden_weights = []
        nodes = self._nodes
        for idx in range(1,len(nodes)):
            hidden_weights.append(0.1 * np.random.randn(nodes[idx -1], nodes[idx]))
            #hidden_weights.append(0.01 *np.random.randn(nodes[idx -1], nodes[idx]))

        return hidden_weights
    
    def _generate_bias(self):
        hidden_layers = []
        nodes = self._nodes
        for i in range(self._n_hidden + 1):
            hidden_layers.append(np.zeros((nodes[i + 1], 1)))
        return hidden_layers
    
    
    def _forward_propagation(self):
        """
        Suppose 2 observations
        
        Suppose previous layer is 3 nodes
        Suppose current layer is 2 nodes
        
        prev shape (2,3)
        prev = ob1 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               ob2 [prev_node_1 val, prev_node_2 val, prev_node_3 val]
               
        layer shape (3,2)
        layer = [weight for current_node_1 for prev_node_1, weight for current_node_2 for prev_node_1]
                [weight for current_node_1 for prev_node_2, weight for current_node_2 for prev_node_2]
                [weight for current_node_1 for prev_node_3, weight for current_node_2 for prev_node_3]
                
        output shape (2,2) # since 2 observations and 2 layers
        output = ob1 [current_node_1 val, current_node_2 val]
                 ob2 [current_node_1 val, current_node_2 val]
                 
        Then for bias in current layer it is (2,1) since 2 nodes in current layer
        
        So for each row in output we add the bias row wise and apply the activation function to each row
        
        prev <- ouput
        
        Move onto next layer...
        """
        if self._batch_size > 0:
            prev = self._batchX
        else:
            prev = self._X
        weights = self._weights
        biases = self._biases
        activations = self._activations[1:-1]
    
        for idx, layer in enumerate(weights):
            if idx == (len(weights) - 1):
                self._forward_inputs.append((prev, None))
                prev = (prev @ layer) + biases[idx].T,
            else:
                weight_output = (prev @ layer) + biases[idx].T
                self._forward_inputs.append((prev, weight_output))
                prev = self._activation(data = weight_output, activation = activations[idx])

        return prev
    
    def _backward_propagation(self, ypred):
            
        j = self._loss_jacobian(ypred)
                
        for i in range(len(self._forward_inputs)-1, -1, -1):
            if i != (len(self._forward_inputs) - 1):
                # activation func on all layers except the last
                der_acti = self._der_activation(self._forward_inputs[i][1])
                j = np.multiply(j,der_acti)

            x = self._forward_inputs[i][0]

            jw = x.T.dot(j)

            b = np.ones((j.shape[0],1))
            jb = j.T.dot(b)
            
            j = j.dot(self._weights[i].T)
            
            self._weights[i] -= self._lr * jw
            self._biases[i] -= self._lr * jb
            
        self._forward_inputs = []        
    
    def _train(self):
        min_loss = old_loss = np.inf
        losses = []
        mses = []
        tol = 0.00001
        terminate_count = anneal_count = step_count = 0
        while True:
            if self._batch_size > 0:
                X_index = np.arange(self._X.shape[0])
                np.random.shuffle(X_index)
                batch_index = X_index[:self._batch_size]
                self._batchX = self._X[batch_index,:]
                self._batchy = self._y[batch_index,:]
            
            batched_out = self._forward_propagation()
            validation_out = self.predict(self._val_X)
                
            loss = self._loss_function(validation_out)
            mse = self._loss_function(validation_out, loss = "mse")
            print("\nloss:")
            print(loss)
            print("mse:")
            print(mse)
            if loss <= min_loss:
                min_loss = loss
                if loss <= (min_loss - 0.1):
                    terminate_count = anneal_count = 0
            if loss <= old_loss:
                anneal_count = 0
            else:
                terminate_count += 1
                anneal_count += 1
                print("INCREASE IN LOSS")
                if anneal_count >= 2:
                    anneal_count = 0
                    self._lr = self._lr / 2
                    print("Decreasing learning rate. New rate is " + str(self._lr))
                if terminate_count > 10:
                    break
            if step_count > 35:
                self._lr = self._lr * 0.9
                print("Annealing learning rate. New rate is " + str(self._lr))
                step_count = 0
            if self._lr < tol:
                break
            
            losses.append(loss)
            mses.append(mse)
            self._backward_propagation(batched_out[0])
            old_loss = loss
            step_count += 1
            
        return losses, mses

In [257]:
# 3x102 (batch = 0)
batch = 0
width = 102
depth = 3

INPUT_SIZE = X_train_normalize.shape[1]
OUTPUT_SIZE = 1
LEARNING_RATE = 0.01

nodes = [INPUT_SIZE] + [width for i in range(depth)] + [OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]
nn = NeuralNetwork()

losses, mses = nn.fit(X = X_train_normalize,
                      y = y_train_normalize,
                      n_hidden = len(nodes) - 2,
                      nodes = nodes,
                      activations = activations,
                      lr = LEARNING_RATE,
                      validation_X = X_val_normalize,
                      validation_y = y_val_normalize,
                      batch_size = batch)

# RESULTS
## loss = 0.186
## mse = 0.373


loss:
0.5279052864994849
mse:
1.0558105729989697

loss:
0.521016250080459
mse:
1.042032500160918

loss:
0.5150033009828197
mse:
1.0300066019656393

loss:
0.5096951629035259
mse:
1.0193903258070518

loss:
0.5049551093314032
mse:
1.0099102186628064

loss:
0.5006781785303489
mse:
1.0013563570606978

loss:
0.49677922654543444
mse:
0.9935584530908689

loss:
0.49319462872445946
mse:
0.9863892574489189

loss:
0.4898680339171785
mse:
0.979736067834357

loss:
0.4867552586631451
mse:
0.9735105173262902

loss:
0.48382540189377404
mse:
0.9676508037875481

loss:
0.4810533268993495
mse:
0.962106653798699

loss:
0.47841293717621797
mse:
0.9568258743524359

loss:
0.47588725610619864
mse:
0.9517745122123973

loss:
0.4734617636823416
mse:
0.9469235273646832

loss:
0.4711210671975926
mse:
0.9422421343951852

loss:
0.46885570543751154
mse:
0.9377114108750231

loss:
0.46665720094594876
mse:
0.9333144018918975

loss:
0.46451884688067896
mse:
0.9290376937613579

loss:
0.4624346446596257
mse:
0.9248692893192


loss:
0.3006799777799839
mse:
0.6013599555599678

loss:
0.30007987308907397
mse:
0.6001597461781479

loss:
0.29948822033924455
mse:
0.5989764406784891

loss:
0.2989010321630445
mse:
0.597802064326089

loss:
0.2983222896802017
mse:
0.5966445793604034

loss:
0.2977489159555592
mse:
0.5954978319111184

loss:
0.29718252331382483
mse:
0.5943650466276497

loss:
0.2966244452180021
mse:
0.5932488904360042

loss:
0.2960707794579614
mse:
0.5921415589159228

loss:
0.29552382273771466
mse:
0.5910476454754293

loss:
0.2949848911748525
mse:
0.589969782349705

loss:
0.29445057853230233
mse:
0.5889011570646047

loss:
0.29392213501127773
mse:
0.5878442700225555

loss:
0.2934022712854949
mse:
0.5868045425709898

loss:
0.29288591279452664
mse:
0.5857718255890533

loss:
0.2923755780057525
mse:
0.584751156011505

loss:
0.29187393695998876
mse:
0.5837478739199775

loss:
0.2913769517545273
mse:
0.5827539035090546

loss:
0.290885852657191
mse:
0.581771705314382

loss:
0.2904022832726305
mse:
0.58080456654526


loss:
0.2617068934038655
mse:
0.523413786807731

loss:
0.26163581925253404
mse:
0.5232716385050681

loss:
0.2615652193162683
mse:
0.5231304386325366

loss:
0.26149526148812496
mse:
0.5229905229762499

loss:
0.2614258413205326
mse:
0.5228516826410652

loss:
0.2613569838670886
mse:
0.5227139677341772

loss:
0.2612884491050123
mse:
0.5225768982100246

loss:
0.2612205010894603
mse:
0.5224410021789206

loss:
0.26115290268856023
mse:
0.5223058053771205

loss:
0.26108574757785413
mse:
0.5221714951557083

loss:
0.26101962365634845
mse:
0.5220392473126969

loss:
0.2609534319550968
mse:
0.5219068639101936

loss:
0.26088754123560626
mse:
0.5217750824712125

loss:
0.26082220183329846
mse:
0.5216444036665969

loss:
0.26075724457389265
mse:
0.5215144891477853

loss:
0.26069262436660945
mse:
0.5213852487332189

loss:
0.26062839893559225
mse:
0.5212567978711845

loss:
0.2605645262847853
mse:
0.5211290525695706

loss:
0.2605009817368383
mse:
0.5210019634736766

loss:
0.26043799446595317
mse:
0.5208759


loss:
0.253942950845537
mse:
0.507885901691074

loss:
0.25390731854642906
mse:
0.5078146370928581

loss:
0.25387171128409197
mse:
0.5077434225681839

loss:
0.2538362057051502
mse:
0.5076724114103004

loss:
0.2538006378321327
mse:
0.5076012756642654

loss:
0.253765149057217
mse:
0.507530298114434

loss:
0.25372983290520523
mse:
0.5074596658104105

loss:
0.2536945430876918
mse:
0.5073890861753836

loss:
0.25365940662684505
mse:
0.5073188132536901

loss:
0.2536246466312393
mse:
0.5072492932624786

loss:
0.2535899644628923
mse:
0.5071799289257846

loss:
0.25355533555390286
mse:
0.5071106711078057

loss:
0.25352077684740637
mse:
0.5070415536948127

loss:
0.25348633714346774
mse:
0.5069726742869355

loss:
0.25345194200802645
mse:
0.5069038840160529

loss:
0.25341764690603413
mse:
0.5068352938120683

loss:
0.25338314241316656
mse:
0.5067662848263331

loss:
0.2533486583329781
mse:
0.5066973166659562

loss:
0.25331414641492694
mse:
0.5066282928298539

loss:
0.25327967853425265
mse:
0.506559357


loss:
0.24923678269951705
mse:
0.4984735653990341

loss:
0.2492125385010754
mse:
0.4984250770021508

loss:
0.24918809106816653
mse:
0.49837618213633306

loss:
0.24916389622675372
mse:
0.49832779245350745

loss:
0.24913944998875429
mse:
0.49827889997750857

loss:
0.2491152777391599
mse:
0.4982305554783198

loss:
0.24909093709540933
mse:
0.49818187419081866

loss:
0.2490667223465725
mse:
0.498133444693145

loss:
0.24904239442258808
mse:
0.49808478884517615

loss:
0.24901823616121885
mse:
0.4980364723224377

loss:
0.24899423794550354
mse:
0.4979884758910071

loss:
0.24896991675885755
mse:
0.4979398335177151

loss:
0.2489458832664092
mse:
0.4978917665328184

loss:
0.2489217270111592
mse:
0.4978434540223184

loss:
0.2488977543114715
mse:
0.497795508622943

loss:
0.24887360807401368
mse:
0.49774721614802736

loss:
0.24884960165378464
mse:
0.4976992033075693

loss:
0.24882569219058231
mse:
0.49765138438116463

loss:
0.24880168433193037
mse:
0.49760336866386073

loss:
0.24877779550291704
mse:

...


loss:
0.23146770351175383
mse:
0.46293540702350766

loss:
0.23146761610587718
mse:
0.46293523221175437

loss:
0.23146752923343214
mse:
0.4629350584668643

loss:
0.2314674428454841
mse:
0.4629348856909682

loss:
0.23146735557919343
mse:
0.46293471115838686

loss:
0.23146726882698396
mse:
0.4629345376539679

loss:
0.23146718155464602
mse:
0.46293436310929204

loss:
0.2314670943743032
mse:
0.4629341887486064

loss:
0.23146700805010886
mse:
0.4629340161002177

loss:
0.23146692138250413
mse:
0.46293384276500826

loss:
0.2314668342696868
mse:
0.4629336685393736

loss:
0.23146674682135554
mse:
0.4629334936427111

loss:
0.2314666605273507
mse:
0.4629333210547014

loss:
0.2314665740535647
mse:
0.4629331481071294

loss:
0.23146648536843106
mse:
0.4629329707368621

loss:
0.23146639859153273
mse:
0.46293279718306546

loss:
0.2314663121104565
mse:
0.462932624220913

loss:
0.2314662257053745
mse:
0.462932451410749

loss:
0.23146613831711926
mse:
0.4629322766342385

loss:
0.2314660512236189
mse:
0.4


loss:
0.23145453024986928
mse:
0.46290906049973857

loss:
0.23145445293359954
mse:
0.46290890586719907

loss:
0.23145437397315932
mse:
0.46290874794631864

loss:
0.2314542958827872
mse:
0.4629085917655744

loss:
0.23145421778769676
mse:
0.4629084355753935

loss:
0.23145414045331028
mse:
0.46290828090662056

loss:
0.23145406137749877
mse:
0.46290812275499754

loss:
0.23145398327312944
mse:
0.4629079665462589

loss:
0.23145390548533817
mse:
0.46290781097067635

loss:
0.23145382692495264
mse:
0.4629076538499053

loss:
0.23145374915334768
mse:
0.46290749830669536

loss:
0.23145367108177978
mse:
0.46290734216355955

loss:
0.23145359258338533
mse:
0.46290718516677065

loss:
0.23145351469921124
mse:
0.4629070293984225

loss:
0.2314534356952439
mse:
0.4629068713904878

loss:
0.23145335726910468
mse:
0.46290671453820936

loss:
0.2314532792311538
mse:
0.4629065584623076

loss:
0.23145320107413725
mse:
0.4629064021482745
Annealing learning rate. New rate is 2.4650347049580723e-05

loss:
0.231453


loss:
0.23144349521153149
mse:
0.46288699042306297

loss:
0.23144343214296415
mse:
0.4628868642859283

loss:
0.23144336882618707
mse:
0.46288673765237415

loss:
0.23144330471065164
mse:
0.4628866094213033

loss:
0.23144324183087658
mse:
0.46288648366175317

loss:
0.2314431788954799
mse:
0.4628863577909598

loss:
0.23144311518122737
mse:
0.46288623036245474

loss:
0.23144305160278983
mse:
0.46288610320557966

loss:
0.2314429884097108
mse:
0.4628859768194216

loss:
0.23144292527029145
mse:
0.4628858505405829

loss:
0.23144286182561546
mse:
0.46288572365123093

loss:
0.23144279819862215
mse:
0.4628855963972443

loss:
0.23144273537042825
mse:
0.4628854707408565

loss:
0.23144267163282375
mse:
0.4628853432656475

loss:
0.23144260861419202
mse:
0.46288521722838405

loss:
0.23144254497638375
mse:
0.4628850899527675

loss:
0.23144248140991655
mse:
0.4628849628198331

loss:
0.23144241825640352
mse:
0.46288483651280704

loss:
0.23144235553660805
mse:
0.4628847110732161

loss:
0.2314422908488825


loss:
0.23143399438701442
mse:
0.46286798877402885

loss:
0.23143393781479632
mse:
0.46286787562959264
Annealing learning rate. New rate is 1.797010299914435e-05

loss:
0.23143388670859807
mse:
0.46286777341719615

loss:
0.23143383516325397
mse:
0.46286767032650794

loss:
0.23143378362993128
mse:
0.46286756725986256

loss:
0.23143373234308656
mse:
0.4628674646861731

loss:
0.2314336810890453
mse:
0.4628673621780906

loss:
0.23143363007204418
mse:
0.46286726014408835

loss:
0.23143357880457452
mse:
0.46286715760914904

loss:
0.2314335272077175
mse:
0.462867054415435

loss:
0.23143347603160955
mse:
0.4628669520632191

loss:
0.23143342486164512
mse:
0.46286684972329023

loss:
0.2314333735621779
mse:
0.4628667471243558

loss:
0.2314333218506445
mse:
0.462866643701289

loss:
0.23143327088116836
mse:
0.4628665417623367

loss:
0.23143321960025245
mse:
0.4628664392005049

loss:
0.2314331680420126
mse:
0.4628663360840252

loss:
0.23143311652005516
mse:
0.4628662330401103

loss:
0.2314330656064

KeyboardInterrupt: 

In [ ]:
# 1x72 (batch = 0)
batch = 0
width = 72
depth = 1

INPUT_SIZE = X_train_normalize.shape[1]
OUTPUT_SIZE = 1
LEARNING_RATE = 0.5

nodes = [INPUT_SIZE] + [width for i in range(depth)] + [OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]
nn = NeuralNetwork()

losses, mses = nn.fit(X = X_train_normalize,
                      y = y_train_normalize,
                      n_hidden = len(nodes) - 2,
                      nodes = nodes,
                      activations = activations,
                      lr = LEARNING_RATE,
                      validation_X = X_val_normalize,
                      validation_y = y_val_normalize,
                      batch_size = batch)

# RESULTS
## loss = 0.186
## mse = 0.373

In [248]:
# 5x72 (batch = 0)
batch = 0
width = 72
depth = 5

INPUT_SIZE = X_train_normalize.shape[1]
OUTPUT_SIZE = 1
LEARNING_RATE = 0.75

nodes = [INPUT_SIZE] + [width for i in range(depth)] + [OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]
nn = NeuralNetwork()

losses, mses = nn.fit(X = X_train_normalize,
                      y = y_train_normalize,
                      n_hidden = len(nodes) - 2,
                      nodes = nodes,
                      activations = activations,
                      lr = LEARNING_RATE,
                      validation_X = X_val_normalize,
                      validation_y = y_val_normalize,
                      batch_size = batch)

# RESULTS
## loss = 0.179
## mse = 0.359


loss:
0.5027851217279157
mse:
1.0055702434558313

loss:
0.49868729367473685
mse:
0.9973745873494737

loss:
0.4954978966323725
mse:
0.990995793264745

loss:
0.4924562340678656
mse:
0.9849124681357312

loss:
0.48891391557776775
mse:
0.9778278311555355

loss:
0.48429286679998107
mse:
0.9685857335999621

loss:
0.47781516793570505
mse:
0.9556303358714101

loss:
0.4685972664200377
mse:
0.9371945328400754

loss:
0.45588353030023815
mse:
0.9117670606004763

loss:
0.43834999274978736
mse:
0.8766999854995747

loss:
0.4146897437634287
mse:
0.8293794875268574

loss:
0.38419234059912416
mse:
0.7683846811982483

loss:
0.34751926029023666
mse:
0.6950385205804733

loss:
0.31056849388696917
mse:
0.6211369877739383

loss:
0.2835456782228202
mse:
0.5670913564456403

loss:
0.2694290200095065
mse:
0.538858040019013

loss:
0.2665869613826832
mse:
0.5331739227653663

loss:
1.0542106478157776
mse:
2.108421295631555
INCREASE IN LOSS

loss:
10.861381237727123
mse:
21.722762475454246
INCREASE IN LOSS
Decreasing


loss:
0.2453216964282987
mse:
0.4906433928565974

loss:
0.24503717663935848
mse:
0.49007435327871696
Annealing learning rate. New rate is 0.09964518750000002

loss:
0.24478412300804195
mse:
0.4895682460160839

loss:
0.2445216488530068
mse:
0.4890432977060136

loss:
0.24426589307342397
mse:
0.48853178614684795

loss:
0.24399803563942662
mse:
0.48799607127885325

loss:
0.24373559550944343
mse:
0.48747119101888686

loss:
0.24336050839942888
mse:
0.48672101679885776

loss:
0.24309585737370165
mse:
0.4861917147474033

loss:
0.24284235143312746
mse:
0.4856847028662549

loss:
0.24257645216861737
mse:
0.48515290433723474

loss:
0.2423231970594845
mse:
0.484646394118969

loss:
0.24205956332141926
mse:
0.4841191266428385

loss:
0.24180695656541498
mse:
0.48361391313082996

loss:
0.24154973077031106
mse:
0.4830994615406221

loss:
0.2412931235921423
mse:
0.4825862471842846

loss:
0.24103189531572672
mse:
0.48206379063145344

loss:
0.24077493254227514
mse:
0.4815498650845503

loss:
0.2405129945579


loss:
0.21464295898646807
mse:
0.42928591797293614

loss:
0.21450861061866056
mse:
0.4290172212373211

loss:
0.21437318058770738
mse:
0.42874636117541476

loss:
0.21423858100812557
mse:
0.42847716201625113
Annealing learning rate. New rate is 0.05295553809018752

loss:
0.2141177182015238
mse:
0.4282354364030476

loss:
0.21399744014381486
mse:
0.4279948802876297

loss:
0.2138791925609722
mse:
0.4277583851219444

loss:
0.21376022491963279
mse:
0.42752044983926557

loss:
0.21364314069667104
mse:
0.42728628139334207

loss:
0.21352517420755862
mse:
0.42705034841511724

loss:
0.21340740166146643
mse:
0.42681480332293287

loss:
0.21328626136874523
mse:
0.42657252273749047

loss:
0.2131659616638819
mse:
0.4263319233277638

loss:
0.21304460535280179
mse:
0.42608921070560357

loss:
0.2129260526601466
mse:
0.4258521053202932

loss:
0.21280499111037862
mse:
0.42560998222075724

loss:
0.21268450436893635
mse:
0.4253690087378727

loss:
0.21256367738847257
mse:
0.42512735477694513

loss:
0.212449262


loss:
0.20254741480058328
mse:
0.40509482960116655

loss:
0.20250921988028386
mse:
0.40501843976056773

loss:
0.20246923499907438
mse:
0.40493846999814875

loss:
0.20242936353256405
mse:
0.4048587270651281

loss:
0.20239108526633107
mse:
0.40478217053266213

loss:
0.2023537610469981
mse:
0.4047075220939962
Annealing learning rate. New rate is 0.02814274411818735

loss:
0.20231854715588177
mse:
0.40463709431176353

loss:
0.20228398072137577
mse:
0.40456796144275153

loss:
0.2022500967853663
mse:
0.4045001935707326

loss:
0.2022153630461275
mse:
0.404430726092255

loss:
0.20218004392118305
mse:
0.4043600878423661

loss:
0.20214610245373632
mse:
0.40429220490747264

loss:
0.2021114363395374
mse:
0.4042228726790748

loss:
0.20207765932263513
mse:
0.40415531864527027

loss:
0.2020439297816957
mse:
0.4040878595633914

loss:
0.20201106122726875
mse:
0.4040221224545375

loss:
0.20197578275312886
mse:
0.4039515655062577

loss:
0.20194170759711974
mse:
0.4038834151942395

loss:
0.20190767992699


loss:
0.19853810480637557
mse:
0.39707620961275114

loss:
0.19851628192458137
mse:
0.39703256384916275

loss:
0.19849393840290191
mse:
0.39698787680580383

loss:
0.19847407917631144
mse:
0.39694815835262287

loss:
0.19845381539965506
mse:
0.39690763079931013

loss:
0.1984332840322704
mse:
0.3968665680645408

loss:
0.19841239234535932
mse:
0.39682478469071863

loss:
0.19839283164038396
mse:
0.3967856632807679
Annealing learning rate. New rate is 0.014956208076913608

loss:
0.1983758317787478
mse:
0.3967516635574956

loss:
0.19835807569479982
mse:
0.39671615138959965

loss:
0.198339857853479
mse:
0.396679715706958

loss:
0.19832059296130972
mse:
0.39664118592261943

loss:
0.19830473750891023
mse:
0.39660947501782046

loss:
0.19828961143984883
mse:
0.39657922287969766

loss:
0.1982748261622265
mse:
0.396549652324453

loss:
0.19826025629352168
mse:
0.39652051258704335

loss:
0.19824380275200845
mse:
0.3964876055040169

loss:
0.19822942166146232
mse:
0.39645884332292464

loss:
0.1982142791

KeyboardInterrupt: 

In [ ]:
# 7x72 (batch = 0)
batch = 0
width = 72
depth = 7

INPUT_SIZE = X_train_normalize.shape[1]
OUTPUT_SIZE = 1
LEARNING_RATE = 3

nodes = [INPUT_SIZE] + [width for i in range(depth)] + [OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]
nn = NeuralNetwork()

losses, mses = nn.fit(X = X_train_normalize,
                      y = y_train_normalize,
                      n_hidden = len(nodes) - 2,
                      nodes = nodes,
                      activations = activations,
                      lr = LEARNING_RATE,
                      validation_X = X_val_normalize,
                      validation_y = y_val_normalize,
                      batch_size = batch)

# RESULTS
## loss = 0.212
## mse = 0.424

In [ ]:
# 8x72 (batch = 0)
batch = 0
width = 72
depth = 8

INPUT_SIZE = X_train_normalize.shape[1]
OUTPUT_SIZE = 1
LEARNING_RATE = 3

nodes = [INPUT_SIZE] + [width for i in range(depth)] + [OUTPUT_SIZE]
activations = ["relu" for i in range(len(nodes))]
nn = NeuralNetwork()

losses, mses = nn.fit(X = X_train_normalize,
                      y = y_train_normalize,
                      n_hidden = len(nodes) - 2,
                      nodes = nodes,
                      activations = activations,
                      lr = LEARNING_RATE,
                      validation_X = X_val_normalize,
                      validation_y = y_val_normalize,
                      batch_size = batch)

# RESULTS
## loss = 0.192
## mse = 0.383